In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
import nltk
from nltk.corpus import stopwords
import string
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

In [2]:
file = open('cooking.stackexchange.txt', 'r')
text = file.readlines()
file.close()

In [3]:
text[0]

'__label__sauce __label__cheese How much does potato starch affect a cheese sauce recipe?\n'

In [4]:
text[0].split("__label__")

['',
 'sauce ',
 'cheese How much does potato starch affect a cheese sauce recipe?\n']

From above we can see that the first item in the list is empty and second item contains label and the third item contains sentence

In [5]:
sentences=[]                 #To store all the sentences
all_label=set()              #To store all the unique labels
labels=[]                    #To store the corresponding label of a sentence 

for i in tqdm(text):         #Separating labels and sentences of each line
    p=i.strip().split("__label__")
    p.pop(0)
    k=p.pop().split(" ")     #k contains sentence
    p.append(k.pop(0))       #p contains all the labels 
    
    for i in range(len(p)):
        p[i]=p[i].strip()
        all_label.add(p[i])
    sentences.append(''.join([i.lower()+" " for i in k]))
    labels.append(p)
all_label=list(all_label)

100%|████████████████████████████████████████████████████████████████████████| 15404/15404 [00:00<00:00, 119950.95it/s]


Preprocessing the sentences

In [6]:
def text_process(mess):
    
    nopunc = [char for char in mess if char not in string.punctuation]#removing punctuations
    nopunc = ''.join(nopunc)
    
    #Removing stopwords and returning the list of words in a sentence without punctuation and stopwords
    return [word.lower() for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [7]:
for i in tqdm(range(len(sentences))):
    sentences[i]=(text_process(sentences[i]))

100%|███████████████████████████████████████████████████████████████████████████| 15404/15404 [00:47<00:00, 326.47it/s]


Creating a list of unique words in all sentences

In [8]:
total_words=set()
for sent in sentences:
    for t in sent:
        total_words.add(t)
print(len(total_words))
total_words=list(total_words)

9469


Creating BAG OF WORDS vectors for all sentences

In [9]:
BOW=[]
for sent in tqdm(sentences):
    sentence_bow_vector=[]
    for t in total_words:
        if t in sent:
            sentence_bow_vector.append(sent.count(t))
        else:
            sentence_bow_vector.append(0)
    BOW.append(sentence_bow_vector)
BOW=np.array(BOW)

100%|███████████████████████████████████████████████████████████████████████████| 15404/15404 [00:37<00:00, 406.44it/s]


In [10]:
BOW.shape

(15404, 9469)

Converting the labels into onehot encoded vectors

In [11]:
label_index={j:i for i,j in enumerate(all_label)}
label_onehot=np.zeros([len(labels),736])
for i in tqdm(range(len(labels))):
    for k in labels[i]:
        label_onehot[i][label_index[k]]=1

100%|████████████████████████████████████████████████████████████████████████| 15404/15404 [00:00<00:00, 260730.67it/s]


In [12]:
label_onehot.shape

(15404, 736)

Normalizing BOW vectors

In [13]:
BOW=preprocessing.normalize(BOW)
BOW=BOW.astype(np.float32)

Applying principal component analysis for dimensionality reduction

In [14]:
pca=PCA(n_components=500)
pca.fit(BOW)
BOW=pca.transform(BOW)

In [15]:
BOW.shape

(15404, 500)

Test train split

In [16]:
sent_train,sent_test,label_train,label_test=train_test_split(BOW,label_onehot,test_size=0.2)

Linear Regression Model

In [17]:
from sklearn.linear_model import LinearRegression
model_lr = LinearRegression()
model_lr.fit(sent_train,label_train)

LinearRegression()

Predictions given by the linear regression model

In [18]:
predictions=model_lr.predict(sent_test)

Calculating accuracy of the model

In [19]:
correct=0
for i in range(len(predictions)):
    k=np.argmax(predictions[i])
    if label_test[i][k]==1:
        correct+=1
print(100*correct/len(predictions))

65.04381694255112


Keras Model

In [20]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout

model = Sequential()

model.add(Dense(500, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=736, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam')

In [21]:
early_stopping_callback = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

In [22]:
history = model.fit(x=sent_train,y=label_train,epochs=100,batch_size=256,validation_data=(sent_test,label_test), callbacks=[early_stopping_callback])

Epoch 1/100
49/49 [==============================] - 1s 14ms/step - loss: 0.3927 - val_loss: 0.0223
Epoch 2/100
49/49 [==============================] - 1s 10ms/step - loss: 0.0202 - val_loss: 0.0189
Epoch 3/100
49/49 [==============================] - 1s 11ms/step - loss: 0.0187 - val_loss: 0.0184
Epoch 4/100
49/49 [==============================] - 1s 11ms/step - loss: 0.0185 - val_loss: 0.0183
Epoch 5/100
49/49 [==============================] - 1s 12ms/step - loss: 0.0184 - val_loss: 0.0183
Epoch 6/100
49/49 [==============================] - 1s 11ms/step - loss: 0.0184 - val_loss: 0.0182
Epoch 7/100
49/49 [==============================] - 1s 11ms/step - loss: 0.0183 - val_loss: 0.0182
Epoch 8/100
49/49 [==============================] - 1s 11ms/step - loss: 0.0183 - val_loss: 0.0182
Epoch 9/100
49/49 [==============================] - 1s 11ms/step - loss: 0.0183 - val_loss: 0.0182
Epoch 10/100
49/49 [==============================] - 1s 11ms/step - loss: 0.0183 - val_loss: 0.0182

49/49 [==============================] - 1s 11ms/step - loss: 0.0100 - val_loss: 0.0120
Epoch 83/100
49/49 [==============================] - 1s 11ms/step - loss: 0.0099 - val_loss: 0.0120
Epoch 84/100
49/49 [==============================] - 1s 11ms/step - loss: 0.0098 - val_loss: 0.0120
Epoch 85/100
49/49 [==============================] - 1s 11ms/step - loss: 0.0097 - val_loss: 0.0120
Epoch 86/100
49/49 [==============================] - 1s 12ms/step - loss: 0.0097 - val_loss: 0.0120


In [23]:
from tensorflow.keras.models import load_model

model.save("my_model")

INFO:tensorflow:Assets written to: my_model\assets


In [24]:
label_pred = model.predict(sent_test)

97/97 [==============================] - 0s 2ms/step


In [25]:
correct_=0
for i in range(len(label_pred)):
    k=np.argmax(label_pred[i])
    if label_test[i][k]==1:
        correct_+=1
print(100*correct_/len(label_pred))

62.34988640051931
